In [1]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import Adam

C:\Users\reza\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [24]:
env = gym.make("MsPacman-v0",render_mode='human')

In [4]:
t1=env.reset()

In [6]:
t1[0]

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[228, 111, 111],
        [228, 111, 111],
        [228, 111, 111],
        ...,
        [228, 111, 111],
        [228, 111, 111],
        [228, 111, 111]],

       [[228, 111, 111],
        [228, 111, 111],
        [228, 111, 111],
        ...,
        [228, 111, 111],
        [228, 111, 111],
        [228, 111, 111]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [8]:
state_size = (88, 80, 1)

In [9]:
action_size = env.action_space.n

In [10]:
color = np.array([210, 164, 74]).mean()
def preprocess_state(state):
    image = state[1:176:2, ::2]
    image = image.mean(axis=2)
    image[image==color] = 0
    image = (image - 128) / 128 - 1
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)
    return image

In [11]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.replay_buffer = deque(maxlen=5000)
        self.gamma = 0.9
        self.epsilon = 0.8
        self.update_rate = 1000
        self.main_network = self.build_network()
        self.target_network = self.build_network()
        self.target_network.set_weights(self.main_network.get_weights())
    def build_network(self):
        model = Sequential()
        
        model.add(Conv2D(32, (8, 8), strides=4, padding='same', input_shape=self.state_size))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (4, 4), strides=2, padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3), strides=1, padding='same'))
        model.add(Activation('relu'))
        
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam())
        return model
    def store_transistion(self, state, action,reward, next_state, done):
        self.replay_buffer.append((state, action,reward, next_state, done))
    
    def epsilon_greedy(self, state):
        if random.uniform(0,1) < self.epsilon:
            return np.random.randint(self.action_size)
        Q_values = self.main_network.predict(state)
        return np.argmax(Q_values[0])
    
    def train(self, batch_size):
        minibatch = random.sample(self.replay_buffer, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if not done:
                target_Q = (reward + self.gamma * np.amax(self.target_network.predict(next_state)))
            else:
                target_Q = reward
            Q_values = self.main_network.predict(state)
            Q_values[0][action] = target_Q
            self.main_network.fit(state, Q_values, epochs=1,verbose=0)
    def update_target_network(self):
        self.target_network.set_weights(self.main_network.get_weights())

In [27]:
num_episodes = 500
num_timesteps = 20000
batch_size = 8
num_screens = 4
dqn = DQN(state_size, action_size)

done = False

time_step = 0
for i in range(num_episodes):
    Return = 0
    state = preprocess_state((env.reset()[0]))
    for t in range(num_timesteps):
        #env.reset()
        env.render()
        time_step += 1
        if time_step % dqn.update_rate == 0:
            dqn.update_target_network()
        action = dqn.epsilon_greedy(state)
        next_state, reward, done,inf,_= env.step(action)
        next_state = preprocess_state(next_state)
        dqn.store_transistion(state, action, reward, next_state, done)
        state = next_state
        Return += reward
        if done:
            print('Episode: ',i, ',' 'Return', Return)
            break
        if len(dqn.replay_buffer) > batch_size:
            dqn.train(batch_size)
            

1/1 [==============================] - 0s 113ms/step


1/1 [==============================] - 0s 82ms/step


1/1 [==============================] - 0s 113ms/step


1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 86ms/step


...

1/1 [==============================] - 0s 112ms/step


1/1 [==============================] - 0s 123ms/step


1/1 [==============================] - 0s 143ms/step
Episode:  0 ,Return 160.0
1/1 [==============================] - 0s 176ms/step


1/1 [==============================] - ETA: 0s

KeyboardInterrupt: 